# Data IMport

In [11]:
import numpy as np
import string
import matplotlib.pyplot as plt
import csv
import pandas as pd

In [12]:
def normalize_words(text):
    words = text.split()
    norm_words = []
    for word in words:
        lastChar = len(word) - 1 
        if word[0] in ("#","@","(","$","'", '"') or word[0].isdigit():
            continue
        if word[lastChar].isdigit() or (word[lastChar] in string.punctuation):
            continue
        norm_words.append(word.lower().rstrip(string.punctuation))
    return norm_words

In [13]:
fulltxt = []
for x in range(85):
    filename = '/data/federalist/' + str(x+1) + '.txt'
    with open(filename, 'r') as myfile:
        data = myfile.read().replace('\n', ' ')
        data = normalize_words(data)
        
    fulltxt = fulltxt + data

In [14]:
vocab = {}
for word in fulltxt:
    if word not in vocab:
        vocab[word] = 1
    else:
        vocab[word] += 1

In [15]:
df = pd.Series(vocab)
df=df[df>25]

In [16]:
authors=pd.read_csv("/data/federalist/authorship.csv")

In [17]:
DF=pd.DataFrame(columns=df.index)

In [18]:
for x in range(85):
    filename = '/data/federalist/' + str(x+1) + '.txt'
    with open(filename, 'r') as myfile:
        data = myfile.read().replace('\n', ' ')
        data = normalize_words(data)
        
    article = pd.DataFrame(columns = df.index)
    article.loc[x] = 0
    for word in data:
        if word not in df.index:
            continue
        article[word] = article[word] + 1
        
    DF = DF.append(article)
DF.index = range(1,len(DF)+1)

# Data Wrangling

In [19]:
DF=DF.reset_index()

In [20]:
DF.head()

,index,a,able,about,absolute,according,act,acts,actual,addition,...,whose,will,with,within,without,would,years,yet,york,you
0,1,25.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,23.0,6.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0
1,2,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,13.0,0.0,2.0,5.0,0.0,3.0,0.0,0.0
2,3,13.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,24.0,10.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0
3,4,16.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,...,1.0,14.0,12.0,0.0,2.0,14.0,0.0,0.0,0.0,1.0
4,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,11.0,1.0,1.0,35.0,1.0,3.0,0.0,1.0


In [21]:
DF=DF.rename(index=str,columns={"index": "Paper"})

In [22]:
DF=DF.merge(authors,how='inner')

In [23]:
sums=DF.groupby("Author").mean()

In [24]:
sumstrans=sums.T
sumtrans=sumstrans.drop('Paper',axis=0)

In [25]:
unknown=DF[pd.isnull(DF['Author'])]
known=DF[~pd.isnull(DF['Author'])]
unknown.head()

,Paper,a,able,about,absolute,according,act,acts,actual,addition,...,will,with,within,without,would,years,yet,york,you,Author
17,18,37.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,1.0,12.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,NaN
18,19,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,20.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,NaN
19,20,28.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,...,3.0,8.0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,NaN
48,49,33.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,11.0,2.0,2.0,21.0,0.0,0.0,0.0,0.0,NaN
49,50,14.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7.0,5.0,6.0,3.0,11.0,0.0,0.0,0.0,0.0,NaN


In [26]:
unknown=unknown.drop("Author",axis=1).set_index("Paper").T
known=known.drop("Author",axis=1).set_index("Paper").T

In [27]:
knowns=DF[~pd.isnull(DF['Author'])]

In [28]:
knowns=knowns.set_index("Paper")
knowns['AuthorNumeric']=1
knowns.loc[knowns['Author']=='Madison','AuthorNumeric']=2
knowns.loc[knowns['Author']=='Jay','AuthorNumeric']=3


In [29]:
knowns.head()

,a,able,about,absolute,according,act,acts,actual,addition,additional,...,with,within,without,would,years,yet,york,you,Author,AuthorNumeric
Paper,,,,,,,,,,,,,,,,,,,,,
1,25.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,6.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0,Hamilton,1
2,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13.0,0.0,2.0,5.0,0.0,3.0,0.0,0.0,Jay,3
3,13.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,Jay,3
4,16.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,12.0,0.0,2.0,14.0,0.0,0.0,0.0,1.0,Jay,3
5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,1.0,1.0,35.0,1.0,3.0,0.0,1.0,Jay,3


# Regression

In [32]:
from sklearn.linear_model import LinearRegression

In [33]:
model=LinearRegression()
model.fit(sumtrans[['Jay','Hamilton','Madison']],unknown[18])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [34]:
model.coef_

array([-0.07121786, -0.56081371,  1.27107459])

In [38]:
# Predicting the unknown papers
for j in unknown.columns:
    print(j)
    model.fit(sumtrans[['Hamilton','Jay','Madison']],unknown[j])
    print(model.coef_)
    print([j/sum([max(0,i/sum(model.coef_ )) for i in model.coef_]) for j in [max(0,i/sum(model.coef_ )) for i in model.coef_]])

18
[-0.56081371 -0.07121786  1.27107459]
[0.0, 0.0, 1.0]
19
[-0.34928372  0.09161397  0.95511578]
[0.0, 0.087523994015241122, 0.91247600598475875]
20
[-0.11694914  0.12301544  0.52337235]
[0.0, 0.19031213541602918, 0.80968786458397091]
49
[ 0.16880277 -0.07475354  0.52525647]
[0.24321089390787035, 0.0, 0.75678910609212968]
50
[-0.03070287  0.0965857   0.34630161]
[0.0, 0.21808187942418358, 0.78191812057581633]
51
[ 0.02641425 -0.21814016  0.78383491]
[0.032600156849751379, 0.0, 0.96739984315024863]
52
[ 0.26574421 -0.14310385  0.51218737]
[0.34160358116658307, 0.0, 0.65839641883341693]
53
[ 0.26510006  0.08827219  0.48286135]
[0.31701675407221747, 0.10555924874964857, 0.57742399717813409]
54
[ 0.25590081 -0.17825776  0.60274525]
[0.2980282823316423, 0.0, 0.70197171766835775]
55
[ 0.53480484  0.05624338  0.24223033]
[0.6418079993995911, 0.067496493781259631, 0.29069550681914919]
56
[ 0.29123313  0.07798419  0.30668463]
[0.43088074384997715, 0.11537795842725104, 0.45374129772277177]
57
[

In [40]:
# Predicting the known papers
for j in known.columns:
    print(j)
    model.fit(sumtrans[['Hamilton','Jay','Madison']],known[j])
    print(model.coef_)
    print([j/sum([max(0,i/sum(model.coef_ )) for i in model.coef_]) for j in [max(0,i/sum(model.coef_ )) for i in model.coef_]])

1
[ 0.70897823  0.16095975 -0.09400813]
[0.81497560441388128, 0.18502439558611866, 0.0]
2
[ -2.09350917e-05   9.14374279e-01   5.28353095e-02]
[0.0, 0.94537346391645616, 0.054626536083543988]
3
[ 0.03345922  0.77834283  0.01090834]
[0.040669499386534497, 0.94607147031320893, 0.013259030300256523]
4
[-0.14591144  1.17433278 -0.01789665]
[0.0, 1.0, 0.0]
5
[ 0.14802777  0.97053536 -0.24011422]
[0.13233742749126712, 0.86766257250873291, 0.0]
6
[ 0.59724056  0.20608698  0.19943642]
[0.5955943569935489, 0.20551893553325332, 0.19888670747319781]
7
[ 1.03928852 -0.11452092  0.03057101]
[0.97142521569943319, 0.0, 0.028574784300566852]
8
[ 0.96343842  0.17844735 -0.17997071]
[0.84372574697409819, 0.15627425302590175, 0.0]
9
[ 0.90311373 -0.07874026  0.01663123]
[0.98191756148414056, 0.0, 0.018082438515859431]
10
[-0.04355953  0.56934486  0.76523478]
[0.0, 0.42660988190623006, 0.57339011809377005]
11
[ 1.46874864  0.28176742 -0.46927905]
[0.83903751472705734, 0.16096248527294257, 0.0]
12
[ 0.9643